In [7]:
import carla
import os
import csv
import math
import time
import numpy as np

# Initialize latest image path
disk_path = 'G:/carla_data_T4_cropped/'
# For simulation in clear roads
output_folder = 'CCW151'
os.makedirs(os.path.join(disk_path, output_folder), exist_ok=True)
# For simulation with traffic
# output_folder = 'output_with_traffic'
frame_id = 0
starting_time = 0
# Initialize the last saved time globally
last_save_time = time.time()

# Function to save data to CSV
def save_to_csv(data, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(data)

# Callback function to capture image path
def on_image(image):
    global disk_path, frame_id, output_folder, last_save_time
    current_time = time.time()

    # Only save the image and data every 1 second
    if current_time - last_save_time >= 0.5:
        #print(f"Saving frame {frame_id}...")  # Add this to check if it's called
        velocity_x = vehicle.get_velocity().x
        velocity_y = vehicle.get_velocity().y
        velocity_z = vehicle.get_velocity().z
        velocity = math.sqrt(velocity_x**2 + velocity_y**2 + velocity_z**2)
        
        control = vehicle.get_control()
        throttle = control.throttle
        steer = control.steer
        brake = control.brake
        hand_brake = control.hand_brake
        reverse = control.reverse
        manual_gear_shift = control.manual_gear_shift
        gear = control.gear
        # vehicle.apply_control(carla.VehicleControl(throttle=0.5, steer=-0.01, brake=0.0, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0))
        # print(control)
        save_image_path = disk_path + output_folder +'/%d.png' % frame_id
        image.save_to_disk(save_image_path)
        
        # Save data to CSV
        data = [frame_id, current_time-starting_time,
                velocity_x, velocity_y, velocity_z, velocity, 
                throttle, steer, brake, hand_brake, reverse, manual_gear_shift, gear,
                save_image_path]
        save_to_csv(data, csv_filename)
        ## manual control to be add here for the car (steering, throttle, break ,. . . .)
        
        frame_id = frame_id + 1
        # Update the last save time
        last_save_time = current_time
    
# Function to move spectator with car
def move_with_the_car():
    spectator = world.get_spectator()
    # Here the spectator is being positioned relative to the position of the car - Relative coordinate system
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-6,z=2)), 
                                vehicle.get_transform().rotation)
    spectator.set_transform(transform)

# Connect to the Carla server
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)

# Load the world
world = client.get_world()
map = world.get_map()

# Load the blueprint library
blueprint_library = world.get_blueprint_library()

# Change weather
weather = world.get_weather()
weather.fog_density = 0
weather.cloudiness = 100
world.set_weather(weather)

# Spawn a vehicle
vehicle_bp = blueprint_library.filter('vehicle.lincoln.mkz_2020')[0]
spawn_points = map.get_spawn_points()
spawn_point = spawn_points[151]
shifted_spawn_point = carla.Transform(spawn_point.transform(carla.Location(x = 50, y = 0)), 
                                      spawn_point.rotation)
vehicle = world.spawn_actor(vehicle_bp, shifted_spawn_point)

# Define the destination point
#destination_waypoint = map.get_waypoint(map.get_spawn_points()[0].location) 

# Attach a camera to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.2, z=1.5))
camera_bp.set_attribute("image_size_x", "640")
camera_bp.set_attribute("image_size_y", "480")
camera_bp.set_attribute("fov", "90")
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Open CSV file for saving data
#csv_filename = output_folder+'_data'+str(time.time())+'.csv'
csv_filename = output_folder+'_data'+'.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Frame id', 'Time(s)', 
                     'Velocity (x)', 'Velocity(y)', 'Velocity(z)', 'Velocity',
                     'Throttle', 'Steer', 'Brake', 'Handbrake', 'Reverse', 'Manual Gear Shift', 'Gear',
                     'Image Path'])

starting_time = time.time()
time.sleep(0.5)

# Set camera settings and attach the callback
camera.listen(lambda image: on_image(image))

# Enable autopilot
vehicle.set_autopilot(True)
# vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0, brake=0.0, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0))

# Define the duration in seconds
duration = 30  # Run for t seconds

try:
    #while time.time() - starting_time < duration:
    #while distance_to_destination < 10.0:
    while True:
        vehicle_location = vehicle.get_location()
        move_with_the_car()  # Move the spectator with the vehicle

        # Check if the vehicle has reached the destination
        #distance_to_destination = vehicle_location.distance(destination_waypoint.transform.location)
        #print(f"Distance to destination: {distance_to_destination:.2f} meters")

        # If the vehicle is within a small threshold of the destination, stop the autopilot
        #if distance_to_destination < 5.0:  # You can adjust the threshold as needed
        #    print("Destination reached!")
        #    vehicle.set_autopilot(False)
        #    break

        time.sleep(0.1)    
    camera.stop()

except:
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

finally:
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [ ]:
# Remove all traffic lights and stop signs
for actor in world.get_actors():
    if 'traffic_light' in actor.type_id or 'traffic_sign' in actor.type_id:
        actor.destroy()

print("All traffic lights and stop signs removed.")

In [ ]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.set_autopilot(True)

In [ ]:
starting_time = time.time()
time.sleep(1)
capturing_time = time.time()
capturing_time-starting_time

In [ ]:
a = a.reshape(480,640,4)

In [ ]:
a.shape

In [ ]:
# Convert BGRA to RGB.
a = a[:, :, :3]
a = a[:, :, ::-1]
a.shape

In [ ]:
from PIL import Image as im 
data = im.fromarray(a)
data

In [ ]:
import carla
import os
import csv
import time

# Initialize latest image path
latest_image_path = None
k = 0
starting_time = time.time()
# Function to save data to CSV
def save_to_csv(data, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(data)

# Callback function to capture image path
def on_image(image):
    global latest_image_path
    spectator = world.get_spectator()
    # Here the spectator is being positioned relative to the position of the car - Relative coordinate system
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-6, z=2)), 
                                vehicle.get_transform().rotation)
    spectator.set_transform(transform)
    
# Function to move spectator with car
def move_with_the_car():
    spectator = world.get_spectator()
    # Here the spectator is being positioned relative to the position of the car - Relative coordinate system
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-6,z=2)), 
                                vehicle.get_transform().rotation)
    spectator.set_transform(transform)

# Connect to the Carla server
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)

# Load the world
world = client.get_world()

# Set synchronous mode settings
settings = world.get_settings()
settings.synchronous_mode = False
settings.fixed_delta_seconds = None
world.apply_settings(settings)

# Load the blueprint library
blueprint_library = world.get_blueprint_library()

# Spawn a vehicle
vehicle_bp = blueprint_library.filter('vehicle.lincoln.mkz_2020')[0]
spawn_points = world.get_map().get_spawn_points()
spawn_point = spawn_points[0]
shifted_spawn_point = carla.Transform(spawn_point.transform(carla.Location(x = 0)), 
                                      spawn_point.rotation)
vehicle = world.spawn_actor(vehicle_bp, shifted_spawn_point)

# Attach a camera to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=0.5, z=1.5))
camera_bp.set_attribute("image_size_x", "640")
camera_bp.set_attribute("image_size_y", "480")
camera_bp.set_attribute("fov", "90")
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Set camera settings and attach the callback
camera.listen(lambda image: on_image(image))

# Enable autopilot
vehicle.set_autopilot(True)

# Define the duration in seconds
duration = 30  # Run for t seconds

# Get the start time
start_time = time.time()
starting_time = time.time()
try:
    while time.time() - start_time < duration:
        pass

except:
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

finally:
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [ ]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [ ]:
import numpy as np

def encode_labels(steering, throttle, steering_range=(-1, 1), throttle_range=(0, 1), num_steering_buckets=10, num_throttle_buckets=10):
    """
    Encode steering and throttle values into a single label.
    
    Parameters:
    steering (float): Steering value between -1 and 1.
    throttle (float): Throttle value between 0 and 1.
    steering_range (tuple): The range of steering values.
    throttle_range (tuple): The range of throttle values.
    num_steering_buckets (int): The number of discrete buckets for steering.
    num_throttle_buckets (int): The number of discrete buckets for throttle.
    
    Returns:
    int: Encoded label.
    """
    # Normalize and scale steering
    steering_min, steering_max = steering_range
    steering_scaled = (steering - steering_min) / (steering_max - steering_min) * (num_steering_buckets - 1)
    steering_encoded = int(np.round(steering_scaled))
    
    # Normalize and scale throttle
    throttle_min, throttle_max = throttle_range
    throttle_scaled = (throttle - throttle_min) / (throttle_max - throttle_min) * (num_throttle_buckets - 1)
    throttle_encoded = int(np.round(throttle_scaled))
    
    # Combine the two encoded values into a single label
    label = steering_encoded * num_throttle_buckets + throttle_encoded
    return label

def decode_labels(label, num_throttle_buckets=10):
    """
    Decode a label back into steering and throttle values.
    
    Parameters:
    label (int): Encoded label.
    num_throttle_buckets (int): The number of discrete buckets for throttle.
    
    Returns:
    tuple: Decoded (steering, throttle) values.
    """
    steering_encoded = label // num_throttle_buckets
    throttle_encoded = label % num_throttle_buckets
    return steering_encoded, throttle_encoded

# Example usage
steering_value = 0.5  # Example steering value
throttle_value = 0.8  # Example throttle value

encoded_label = encode_labels(steering_value, throttle_value)
print(f"Encoded label: {encoded_label}")

# Decode to verify
decoded_steering, decoded_throttle = decode_labels(encoded_label)
print(f"Decoded steering: {decoded_steering}, Decoded throttle: {decoded_throttle}")


In [ ]:
def decode_labels(label, num_throttle_buckets=10, steering_range=(-1, 1), throttle_range=(0, 1)):
    """
    Decode a label back into steering and throttle values.
    
    Parameters:
    label (int): Encoded label.
    num_throttle_buckets (int): The number of discrete buckets for throttle.
    steering_range (tuple): The range of steering values.
    throttle_range (tuple): The range of throttle values.
    
    Returns:
    tuple: Decoded (steering, throttle) values in their original ranges.
    """
    steering_encoded = label // num_throttle_buckets
    throttle_encoded = label % num_throttle_buckets
    
    steering_min, steering_max = steering_range
    throttle_min, throttle_max = throttle_range
    
    steering = steering_encoded / (num_throttle_buckets - 1) * (steering_max - steering_min) + steering_min
    throttle = throttle_encoded / (num_throttle_buckets - 1) * (throttle_max - throttle_min) + throttle_min
    
    return steering, throttle

# Example usage
steering_value = 0.5  # Example steering value
throttle_value = 0.8  # Example throttle value

encoded_label = encode_labels(steering_value, throttle_value)
print(f"Encoded label: {encoded_label}")

# Decode to verify
decoded_steering, decoded_throttle = decode_labels(encoded_label)
print(f"Decoded steering: {decoded_steering}, Decoded throttle: {decoded_throttle}")